In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

### 1. Loading the dataset

In [100]:
pd.read_excel("../dataset/premiums_young_with_gr.xlsx")
df.head()

NameError: name 'df' is not defined

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.replace(' ', '_').str.lower() ## snake case convention and removing spaces
df.head()

### 2. EDA & data cleaning

#### i. Handling na values

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace = True) ## very few na as no of rows = 50k. we can just drop them.
df.isna().sum()

#### ii. Handling duplicates

In [ ]:
df.duplicated().sum() ##no duplicates

In [ ]:
df.drop_duplicates(inplace=True) ## we can till keep the code because if tomorrow the file is updated and it has duplicaytes then we'll have handling code.
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
## max age = 356.000000, min number_of_dependants = -3, max income_lakhs = 930 (9.3 cr) | These are or can be outliers.  

#### iii. Data Cleaning: number_of_dependants

In [ ]:
df[df.number_of_dependants<0].shape ## 73 values have no of dependents as negative

In [ ]:
df['number_of_dependants'][df.number_of_dependants < 0].unique() ## 72 rows have these values. Maybe +ve values converted to negative by mistake.

In [ ]:
df['number_of_dependants'] = abs(df['number_of_dependants'])

In [ ]:
df.number_of_dependants.describe()  # now looking good

#### iii. Data Cleaning: visualizing outliers

In [ ]:
sns.boxplot(x = df['age'])
plt.show()

## Numeric Columns

### Univariate Analysis: Numeric Columns

In [ ]:
## for all numeric columns

In [ ]:
numeric_columns = df.select_dtypes(['float64','int64']).columns
numeric_columns

In [ ]:
for col in numeric_columns:
    plt.figure(figsize=(12, 3)) 
    sns.boxplot(x=df[col])
    plt.show()

### iv. Outlier Treatment: Age Column

In [ ]:
df['age'][df.age > 100].unique() 

In [ ]:
df1 = df[df.age<=100].copy() ## row filtering and then copying into a new df just for safety
df1.describe()  ## outliers gone from age column.

### iv. Outlier Treatment: Income Column

In [ ]:
sns.histplot(df1.income_lakhs)

In [ ]:
df.income_lakhs.quantile([0.25, 0.75])

In [ ]:
def get_iqr_bounds(col):
    Q1, Q3 = col.quantile([0.25, 0.75])
    IQR = Q3-Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

lower, upper = get_iqr_bounds(df1['income_lakhs'])
lower, upper

In [ ]:
## we don't have to worry about the lower bound as anywats our minimun income is 1 lakh which is okay.

In [ ]:
df[df.income_lakhs>upper].shape

In [ ]:
quantile_threshold = df1.income_lakhs.quantile(0.999)
quantile_threshold

In [ ]:
df2 = df1[df1.income_lakhs<=quantile_threshold].copy()
df2.describe()

In [ ]:
## income 1->100 lakhs ->reasonable range

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))  # Adjust the size to ensure plots are not squeezed

for i, column in enumerate(numeric_columns):
    # Locating the correct subplot using integer division and modulus
    ax = axs[i // 3, i % 3]  # Row index is i//3, column index is i%3
    sns.histplot(df2[column], kde=True, ax=ax)
    ax.set_title(column)

# If the last subplot axis is unused, you can turn it off
if len(numeric_columns) % 3 != 0:
    for j in range(len(numeric_columns), 6):  # This will disable any unused subplots
        axs.flat[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
## most income datasets are right skewed

In [ ]:
 sns.histplot(df2.income_lakhs, kde=True)

### Bivariate Analysis: Numeric Columns

In [ ]:
## scatterplot -> annual_premium_amount vs age

sns.scatterplot(df2, x = 'age', y = 'annual_premium_amount')

In [ ]:
## we can se clusters here after observing carefully. 
## Also another observation-> as age increases, premium also increases. COMMON SENSE

In [ ]:
numeric_features = ['age', 'income_lakhs', 'number_of_dependants', 'genetical_risk']

fig, axes = plt.subplots(1, len(numeric_features), figsize=(18, 6))  

for ax, column in zip(axes, numeric_features):
    sns.scatterplot(x=df2[column], y=df2['annual_premium_amount'], ax=ax)
    ax.set_title(f'{column} vs. Annual Premium Amount')
    ax.set_xlabel(column)
    ax.set_ylabel('Annual Premium Amount')

plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
## not much correlation between income, premium and no of dependets, premium

##  Categorical Columns

In [ ]:
categorical_cols = ['gender', 'region', 'marital_status', 'bmi_category', 'smoking_status', 'employment_status', 'income_level', 'medical_history', 'insurance_plan']
for col in categorical_cols:
    print(col, ":", df2[col].unique())

In [ ]:
##smoking_status : ['No Smoking' 'Regular' 'Occasional' 'Smoking=0' 'Does Not Smoke' 'Not Smoking'] -> problems

df2['smoking_status'].replace({
    'Not Smoking': 'No Smoking',
    'Does Not Smoke': 'No Smoking',
    'Smoking=0': 'No Smoking'
}, inplace=True)

df2['smoking_status'].unique()

In [ ]:
### Univariate Analysis: Categorical columns

In [ ]:
pct_count = df2['gender'].value_counts(normalize = True)
pct_count

In [ ]:
sns.barplot(x=pct_count.index, y=pct_count.values)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 18))  
axes = axes.flatten()  # Flatten the 2D array of axes into 1D for easier iteration

for ax, column in zip(axes, categorical_cols):
    # Calculate the percentage distribution of each category
    category_counts = df2[column].value_counts(normalize=True) * 100  # normalize=True gives the relative frequencies
    
    # Plotting the distribution using barplot
    sns.barplot(x=category_counts.index, y=category_counts.values, ax=ax)
    ax.set_title(f'Percentage Distribution of {column}')
    ax.set_ylabel('Percentage of Policyholders (%)')
    ax.set_xlabel(column)  # Set xlabel to the column name for clarity
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()  
plt.show()

### Bivariate Analysis: Categorical columns

In [ ]:
# Cross-tabulation of gender and smoking status
crosstab = pd.crosstab(df2.income_level, df2.insurance_plan)
print(crosstab)
crosstab.plot(kind = 'bar', stacked = True)
plt.title('Income vs Plan')
plt.ylabel('Count')
plt.show()

In [ ]:
sns.heatmap(crosstab, annot=True, cmap='coolwarm',fmt="d")
plt.title('Heatmap of Income vs Plan')
plt.show()

### 3. Feature Engineering

In [ ]:
df2.head()

In [ ]:
df2.medical_history.unique()

In [ ]:
## here we can calculate and assign some kind of risk score becaus ml models can't process this text values.
## more diseases -> more premium

In [ ]:
## Define the risk scores for each condition.
## Logic used here is just an example. In real world, stakeholders decide what to do in situations like these.

risk_scores = {
    "diabetes": 6,
    "heart disease": 8,
    "high blood pressure":6,
    "thyroid": 5,
    "no disease": 0,
    "none":0
}

df2[['disease1', 'disease2']] = df2['medical_history'].str.split(" & ", expand=True).apply(lambda x: x.str.lower())
df2['disease1'].fillna('none', inplace=True)
df2['disease2'].fillna('none', inplace=True)
df2['total_risk_score'] = 0

for disease in ['disease1', 'disease2']:
    df2['total_risk_score'] += df2[disease].map(risk_scores)

# Normalize the risk score to a range of 0 to 1
max_score = df2['total_risk_score'].max()
min_score = df2['total_risk_score'].min()
df2['normalized_risk_score'] = (df2['total_risk_score'] - min_score) / (max_score - min_score)
df2.sample(20)

### Label encoding text columns

In [ ]:
df2.insurance_plan.unique

In [ ]:
df2['insurance_plan'] = df2['insurance_plan'].map({'Bronze': 1, 'Silver': 2, 'Gold': 3})

In [ ]:
df2.income_level.unique()

In [ ]:
df2['income_level'] = df2['income_level'].map({'<10L':1, '10L - 25L': 2, '25L - 40L':3, '> 40L':4})

In [ ]:
df2.head()

### Label encoding text columns

In [ ]:
nominal_cols = ['gender', 'region', 'marital_status', 'bmi_category', 'smoking_status', 'employment_status']
df3 = pd.get_dummies(df2, columns=nominal_cols, drop_first=True, dtype=int)
df3.head(3)

## 3. Feature Selection

In [ ]:
df3.info()

In [ ]:
df4 = df3.drop(['medical_history','disease1', 'disease2', 'total_risk_score'], axis=1)
df4.head(3)                

#### Calculate VIF for Multicolinearity

In [ ]:
## correlation analysis
cm = df4.corr()

plt.figure(figsize=(20,12))
sns.heatmap(cm, annot=True)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
## genetical risk has strong correlation (0.62) with premium amount. If we include this feature in the model training, the performance will likely improve. 

In [ ]:
## feature scaling

X = df4.drop('annual_premium_amount', axis='columns')
y = df4['annual_premium_amount']

from sklearn.preprocessing import MinMaxScaler

cols_to_scale = ['age','number_of_dependants', 'income_level',  'income_lakhs', 'insurance_plan', 'genetical_risk']
scaler = MinMaxScaler()

X[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])
X.describe()

In [ ]:
## calculating VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(data):
    vif_df = pd.DataFrame()
    vif_df['Column'] = data.columns
    vif_df['VIF'] = [variance_inflation_factor(data.values,i) for i in range(data.shape[1])]
    return vif_df

In [ ]:
calculate_vif(X)

In [ ]:
calculate_vif(X.drop('income_level', axis="columns")) ## vif value after droppinh the column with highest vif value looks perfectly fine.

In [ ]:
# we will drop income_lakhs due to high VIF value
X_reduced = X.drop('income_level', axis="columns")
X_reduced

## 4. Model training    

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.30, random_state=10)

# shape of the X_train, X_test, y_train, y_test features
print("x train: ",X_train.shape)
print("x test: ",X_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

#### i. Linear Regression Model

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
test_score = model_lr.score(X_test, y_test)
train_score = model_lr.score(X_train, y_train)
train_score, test_score

In [ ]:
## we get a score of 0.98 after adding the gr feature. without it, it was only .58  

In [ ]:
y_pred = model_lr.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred)
rmse_lr = np.sqrt(mse_lr)
print("Linear Regression ==> MSE: ", mse_lr, "RMSE: ", rmse_lr)

In [ ]:
X_test.shape

In [ ]:
X_test.head(1)

In [ ]:
np.set_printoptions(suppress=True, precision=6)
model_lr.coef_

In [ ]:
feature_importance = model_lr.coef_ ## It tells how much important the features are for the final prediction. In LR, feature importance is simply coefficients.    

# Create a DataFrame for easier handling
coef_df = pd.DataFrame(feature_importance, index=X_train.columns, columns=['Coefficients'])

# Sort the coefficients for better visualization
coef_df = coef_df.sort_values(by='Coefficients', ascending=True)
coef_df

In [ ]:
#Plotting
plt.figure(figsize=(8, 4))
plt.barh(coef_df.index, coef_df['Coefficients'], color='steelblue')
plt.xlabel('Coefficient Value')
plt.title('Feature Importance in Linear Regression')
plt.show()

#### ii. Ridge Regression Model

In [ ]:
model_rg = Ridge(alpha=1)
model_rg.fit(X_train, y_train)
test_score = model_rg.score(X_test, y_test)
train_score = model_rg.score(X_train, y_train)
train_score, test_score

In [ ]:
y_pred = model_rg.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred)
rmse_lr = np.sqrt(mse_lr)
print("Ridge Regression ==> MSE: ", mse_lr, "RMSE: ", rmse_lr)

#### iii. XGBoost 

In [ ]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor(n_estimators=20, max_depth=3)
model_xgb.fit(X_train, y_train)
test_score = model_xgb.score(X_test, y_test)
train_score = model_xgb.score(X_train, y_train)
train_score, test_score

In [ ]:
y_pred = model_xgb.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred)
rmse_lr = np.sqrt(mse_lr)
print("XGBoost Regression ==> MSE: ", mse_lr, "RMSE: ", rmse_lr)

In [ ]:
model_xgb = XGBRegressor()
param_grid = {
    'n_estimators': [20, 40, 50],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
}
random_search = RandomizedSearchCV(model_xgb, param_grid, n_iter=10, cv=3, scoring='r2', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)
random_search.best_score_

In [ ]:
random_search.best_params_

In [ ]:
##both lr and xgb are giving approx same value so we'll select linear regression 

In [ ]:
best_model = model_lr

## 5. Error Analysis

In [ ]:
## We can't just rely on r2 score and deploy the model. We need to find the margin of error for each of the sample we are running our prediction.

In [ ]:
y_pred = best_model.predict(X_test)

residuals = y_pred - y_test
residuals_pct = (residuals / y_test) * 100

results_df = pd.DataFrame({
    'actual': y_test, 
    'predicted': y_pred, 
    'diff': residuals, 
    'diff_pct': residuals_pct
})
results_df.head()

In [ ]:
sns.histplot(results_df.diff_pct, kde = True)
plt.show()

In [ ]:
## There are many records where we are predicting the premium upto 80 percent higher. Basically wrong output.

In [ ]:
extreme_error_threshold = 10  # We can adjust this threshold based on domain knowledge or requirements
extreme_results_df = results_df[np.abs(results_df['diff_pct']) > extreme_error_threshold]
extreme_results_df.head()

In [ ]:
extreme_results_df.shape

In [ ]:
results_df.shape

In [ ]:
extreme_errors_pct = extreme_results_df.shape[0]*100/X_test.shape[0]
extreme_errors_pct

In [ ]:
## Down to 2 percert from 73 percent

In [ ]:
extreme_results_df[abs(extreme_results_df.diff_pct)>50].sort_values("diff_pct",ascending=False).shape ## more than 50 percent error margin in extreme_df

In [ ]:
extreme_results_df[abs(extreme_results_df.diff_pct)>50].sort_values("diff_pct",ascending=False)

In [ ]:
##There will be about 549 customers whom we will overcharge or underchage by more than 50%

In [ ]:
extreme_results_df.index

In [ ]:
extreme_errors_df = X_test.loc[extreme_results_df.index]
extreme_errors_df.head()

## 6. Exporting the model.

In [ ]:
from joblib import dump

dump(best_model, "artifacts/model_young.joblib") ## saving the model
scaler_with_cols = {   ## Our model is training on the scaled data, not the original data so we'll also have to saved this so that during the prediction, the scaling can be done
    'scaler': scaler,
    'cols_to_scale': cols_to_scale
}
dump(scaler_with_cols, "artifacts/scaler_young.joblib")